In [22]:
import pandas as pd
from Indicator import DataManage
from tqdm import tqdm

In [7]:
data = pd.read_csv("BTC_USDT_1m.csv", index_col = 0)
parameter = [
        {"rsi" : {"period" : 50}},
        {"ma" : {"period" : 50}},
        {"ma" : {"period" : 200}},
        {"ema" :{"period" : 50}},
        {"ema" :{"period" : 200}},
        {"stochastic" : {"n" : 50,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 50,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 50, "slow_period" : 200}}
    ]
data = DataManage(data, parameter).get_data()
data.head()

,datetime,open,high,low,close,volume,mean_50,mean_200
199,2017-12-31 18:19:00,13830.00,13900.50,13830.00,13830.02,10.065550,13815.1404,13604.50845
200,2017-12-31 18:20:00,13861.71,13912.57,13829.33,13844.89,10.212564,13816.2954,13607.13290
201,2017-12-31 18:21:00,13844.91,13873.00,13829.33,13844.91,5.798086,13817.9002,13609.79095
202,2017-12-31 18:22:00,13845.03,13875.00,13843.25,13843.25,3.345282,13819.4850,13612.48215
203,2017-12-31 18:23:00,13843.25,13844.87,13829.33,13831.01,2.840560,13820.8232,13614.96720


## 일반 골드크로스 매매법

In [20]:
from tqdm import tqdm
class backtest:
    def __init__(self,data, result_label = None, test_size = None, set_amount = 0.02, fee = 0.0008, max_buy = 1000):
        """
        data(DataFrame) : ex) Include close price

        result_label(list) : ex) [-1,0,0,1,-1,....] (-1,0,1)만 포함

        test_size(int) : ex) 1440 * 30

        set_amount(float) : ex) 0.02

        fee(float) : ex) fee == 0.08% -> 0.0008

        max_buy(int) : ex) 10000 
        """
        self.data = data # 주가 데이터
        self.result_label = result_label # 머신러닝의 예측값
        self.test_size = test_size # 테스트 데이터의 길이
        self.set_amount = set_amount # 한번 매수할 떄 수량
        self.fee = fee # 한번 거래 할 때 수수료
        self.max_buy = max_buy # 최대 누적 매수 횟수

        # 손절형 순환매매에서 사용하는 변수
        self.buy_flag = 0
        self.sell_flag = 0

        # 공통 사용 변수
        self.quantityBuying = 0 # 현재 매수 수량
        self.BuyingList = [] # 현재 매수 했던 가격들의 리스트
        self.totalYield = 0 # 최종 수익률
        self.totalNumberSales = 0 # 총 매도 횟수
        self.win = 0
        self.quantityBuyingList = [] # 매수 수량을 저장하는 리스트
        self.amount = 0 # 현재 가지고 있는 주식 수량
        self.MDDList = [] # MDD를 계산하기 위한 리스트
        self.average_price = 0 # 평단가
        
    def initializes(self):
        self.buy_flag = 0
        self.sell_flag = 0

        # 공통 사용 변수
        self.quantityBuying = 0
        self.BuyingList = []
        self.totalYield = 0
        self.totalNumberSales = 0
        self.win = 0
        self.quantityBuyingList = []
        self.amount = 0
        self.MDDList = []
        self.average_price = 0

    def SellInitializes(self,currYield):
        if currYield > 0:
            self.win += 1
        self.average_price = 0
        self.totalNumberSales += 1
        self.totalYield += currYield
        self.amount = 0
        self.BuyingList.clear()
        self.quantityBuying = 0
        self.buy_flag = 0
        self.sell_flag = 0
        

    def GoldenCross(self):
        self.initializes()
        for i in tqdm(range(1, len(self.data))):
            
            if self.data.iloc[i-1]["mean_50"] < self.data.iloc[i-1]["mean_200"] and self.data.iloc[i]["mean_50"] > self.data.iloc[i]["mean_200"] and self.quantityBuying < self.max_buy:
                self.quantityBuying += 1
                self.average_price = ( self.average_price * self.amount + self.data.iloc[i]['close'] * self.set_amount) / (self.amount + self.set_amount)
                self.amount += self.set_amount
                self.BuyingList.append(self.data.iloc[i]['close'])

            if self.data.iloc[i-1]["mean_50"] > self.data.iloc[i-1]["mean_200"] and self.data.iloc[i]["mean_50"] < self.data.iloc[i]["mean_200"] and self.quantityBuying != 0:
                currYield = (self.data.iloc[i]['close']-self.average_price) * self.amount - (self.average_price * self.fee * self.amount)
                self.quantityBuyingList.append(self.quantityBuying)

                self.SellInitializes(currYield)
                
            if self.average_price > 0:
                if (self.data.iloc[i]["close"] - self.average_price)/self.average_price < -0.03:
                    currYield = (self.data.iloc[i]['close']-self.average_price) * self.amount - (self.average_price * self.fee * self.amount)
                    self.quantityBuyingList.append(self.quantityBuying)

                    self.SellInitializes(currYield)
                if (self.data.iloc[i]["close"] - self.average_price)/self.average_price > 0.05:
                    currYield = (self.data.iloc[i]['close']-self.average_price) * self.amount - (self.average_price * self.fee * self.amount)
                    self.quantityBuyingList.append(self.quantityBuying)

                    self.SellInitializes(currYield)
                
            if self.quantityBuying > 0:
                self.MDDList.append(round((sum(self.BuyingList)/self.quantityBuying-self.data.iloc[i]['close'])/self.data.iloc[i]['close'],2))

            
        self.quantityBuyingList.sort()
        self.MDDList.sort()
        
        if len(self.quantityBuyingList) == 0 or self.totalNumberSales == 0:
            print("Zero DivisionError")
            raise ZeroDivisionError
        return {
            "averageNumberSales" : sum(self.quantityBuyingList)/len(self.quantityBuyingList),
            "totalYield" : self.totalYield,
            "win_rate" : self.win/self.totalNumberSales,
            "MDD" : self.MDDList[-1],
            "max_buying" : self.quantityBuyingList[-1],
            "NumberTrading" : self.totalNumberSales
        }
    

In [21]:
backtest(data[:100000]).GoldenCross()

 11%|█         | 11086/99999 [00:11<01:29, 994.80it/s] /var/folders/5v/ywkv8vl52zs0xz8k0x15y8fm0000gn/T/ipykernel_45804/3762068184.py:89: RuntimeWarning: divide by zero encountered in double_scalars
  if (self.data.iloc[i]["close"] - self.average_price)/self.average_price > 0.05:
100%|██████████| 99999/99999 [01:41<00:00, 985.89it/s] 


{'averageNumberSales': 0.9576271186440678,
 'totalYield': -79.07371935999986,
 'win_rate': 0.3361581920903955,
 'MDD': 0.03,
 'max_buying': 1,
 'NumberTrading': 354}

In [23]:
term = 120
def ck(s):
    global term
    for i in range(1,term):
        if (data.iloc[s + i]["close"] - data.iloc[s]["close"]) / data.iloc[s]["close"] > 0.03:
            return True
        elif (data.iloc[s + i]["close"] - data.iloc[s]["close"]) / data.iloc[s]["close"] < -0.015:
            return False
    return False
def labeling(data):
    global term
    result = []
    for i in tqdm(range(1, len(data))):
        if data.iloc[i - 1]["mean_50"] < data.iloc[i - 1]["mean_200"] and data.iloc[i]["mean_50"] > data.iloc[i]["mean_200"]:
            if ck(i):
                # 매수 가능
                result.append(0)
            else:
                # 매수 불가능
                result.append(1)
    return result

In [24]:
Labeled = labeling(data)

100%|█████████▉| 2713096/2713188 [30:39<00:00, 1474.87it/s]


IndexError: single positional indexer is out-of-bounds